In [22]:
%load_ext autoreload
%autoreload 2

import numpy as np
import matplotlib.pyplot as plt
import os
import sys
import glob

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torchvision
import torchvision.transforms as transforms
from PIL import Image
from PIL import ImageFile

sys.path.append( '../' )
from ConvVAE import ConvVAE
from utils.Dataset import Rollout_Dataset, to_latent

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
VAE_model = ConvVAE(n_channel=3, z_dim=128)
VAE_model.load_state_dict(torch.load("./weights/segmodel_rollouts.pt"))

<All keys matched successfully>

In [3]:
validation_prop = 0.15

s_paths = glob.glob(os.path.join("../S_Rollouts_11/*.npz"))
d_paths = glob.glob(os.path.join("../D_Rollouts_11/*.npz"))
print(f"S_Rollouts lenght: {len(s_paths)}")
print(f"D_Rollouts lenght: {len(d_paths)}")
n_static_examples = len(s_paths)
n_dinamic_examples = len(d_paths)

s_idxs = np.random.choice(n_static_examples-1, size=round(n_static_examples*(1-validation_prop)),
                          replace=False)
s_cidxs = np.array(list(set(np.arange(n_static_examples)) - set(s_idxs)))

d_idxs = np.random.choice(n_dinamic_examples-1, size=round(n_dinamic_examples*(1-validation_prop)),
                          replace=False)
d_cidxs = np.array(list(set(np.arange(n_dinamic_examples))- set(d_idxs)))

print(f"s_idxs lenght: {s_idxs.shape[0]}")
print(f"s_cidxs lenght: {s_cidxs.shape[0]}")

print(f"d_idxs lenght: {d_idxs.shape[0]}")
print(f"d_cidxs lenght: {d_cidxs.shape[0]}")

del s_paths
del d_paths

S_Rollouts lenght: 97
D_Rollouts lenght: 100
s_idxs lenght: 82
s_cidxs lenght: 15
d_idxs lenght: 85
d_cidxs lenght: 15


In [34]:
s_path = "../S_Rollouts_11"
d_path = "../D_Rollouts_11"
seq_len = 3
buffer_size = 3 # number of files
is_complt_states = True
transform = transforms.Compose([transforms.Resize((80, 160)), transforms.ToTensor()])

s_train_dataset = Rollout_Dataset(s_path, s_idxs, seq_len, buffer_size, is_complt_states, transform=transform)
s_val_dataset = Rollout_Dataset(s_path, s_cidxs, seq_len, buffer_size, is_complt_states, transform=transform)

batch_size = 32
s_train_dataloader = DataLoader(s_train_dataset, batch_size=batch_size, shuffle=False)
s_val_dataloader = DataLoader(s_val_dataset, batch_size=batch_size, shuffle=False)

In [38]:
for i, X_tr in enumerate(s_train_dataloader):
    print(X_tr[-2].shape)
    states, actions, rewards, next_states, terminals = X_tr if len(X_tr)==5 else X_tr[:-2]
    complt_states, next_complt_states = X_tr[-2:] if len(X_tr) > 5 else None, None
    print(complt_states[0].shape)
    states = to_latent(VAE_model, states, next_states, complt_states, next_complt_states)
    print(states.shape)
    break
    
    

torch.Size([32, 3, 6])
torch.Size([32, 3, 6])


TypeError: expected np.ndarray (got list)